In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("IPODataFull.csv", low_memory=False,encoding='unicode_escape')
df.head()

,Symbol,DaysBetterThanSP,daysProfit,daysProfitGrouped,exactDiffernce,Year,Month,Day,dayOfWeek,closeDay0,...,usableCEOAge,usableCEOGender,usablePresidentAge,usablePresidentGender,FoundingDateGrouped,yearDifferenceGrouped,Profitable,Safe,HomeRunDay,HomeRun
0,A,122,249,200+,NaN,1999,11,18,3,28.6358,...,Blank,Blank,Blank,Blank,1951 - 2000,Unknown,1,0,77.0,1
1,AAC,131,262,200+,232.0,2014,10,2,3,18.5000,...,40 - 49,male,Blank,Blank,2011 - present,Unknown,1,1,NaN,0
2,AAOI,125,262,200+,6054.0,2013,9,26,3,9.9600,...,Blank,Blank,Blank,Blank,1951 - 2000,Unknown,1,0,NaN,0
3,AAP,128,261,200+,NaN,2001,11,29,3,13.9000,...,Blank,Blank,Blank,Blank,1901 - 1950,Unknown,1,0,NaN,0
4,AAT,123,127,100 - 149,181.0,2011,1,13,3,21.2500,...,Blank,Blank,Blank,Blank,2001 - 2010,Unknown,0,0,NaN,0


In [5]:
df.shape

(3762, 1664)

In [13]:
colnames = list(df.columns)
colnames[1000:len(colnames)]

['highDay198',
 'openDay198',
 'lowDay198',
 'volumeDay198',
 'closeDay199',
 'highDay199',
 'openDay199',
 'lowDay199',
 'volumeDay199',
 'closeDay200',
 'highDay200',
 'openDay200',
 'lowDay200',
 'volumeDay200',
 'closeDay201',
 'highDay201',
 'openDay201',
 'lowDay201',
 'volumeDay201',
 'closeDay202',
 'highDay202',
 'openDay202',
 'lowDay202',
 'volumeDay202',
 'closeDay203',
 'highDay203',
 'openDay203',
 'lowDay203',
 'volumeDay203',
 'closeDay204',
 'highDay204',
 'openDay204',
 'lowDay204',
 'volumeDay204',
 'closeDay205',
 'highDay205',
 'openDay205',
 'lowDay205',
 'volumeDay205',
 'closeDay206',
 'highDay206',
 'openDay206',
 'lowDay206',
 'volumeDay206',
 'closeDay207',
 'highDay207',
 'openDay207',
 'lowDay207',
 'volumeDay207',
 'closeDay208',
 'highDay208',
 'openDay208',
 'lowDay208',
 'volumeDay208',
 'closeDay209',
 'highDay209',
 'openDay209',
 'lowDay209',
 'volumeDay209',
 'closeDay210',
 'highDay210',
 'openDay210',
 'lowDay210',
 'volumeDay210',
 'closeDay211',

In [46]:
close_day_0 = df.columns.get_loc('closeDay0')
volume_day_261 = df.columns.get_loc('volumeDay261')

print(close_day_0)
print(volume_day_261)

df_prices = df.iloc[:,close_day_0:volume_day_261]

colname_symbol = (df.columns[df.columns.get_loc('Symbol')])

df_prices.insert(0, colname_symbol, df['Symbol'])
df_prices

9
1318


,Symbol,closeDay0,highDay0,openDay0,lowDay0,volumeDay0,closeDay1,highDay1,openDay1,lowDay1,...,volumeDay259,closeDay260,highDay260,openDay260,lowDay260,volumeDay260,closeDay261,highDay261,openDay261,lowDay261
0,A,28.6358,33.5207,27.3725,30.6572,59753154.0,27.2040,28.9727,26.8253,28.9323,...,2557941.0,28.0462,29.4363,27.7936,28.9323,2944860.0,31.9205,31.9629,29.6465,29.6465
1,AAC,18.5000,20.1000,17.6000,20.0000,2799073.0,18.6500,18.7500,18.0500,18.2000,...,419099.0,22.5900,22.6900,22.1100,22.2700,197468.0,24.4300,24.4800,22.4200,22.4200
2,AAOI,9.9600,10.0900,9.3700,10.0000,948999.0,10.1000,10.4400,10.0000,10.4400,...,167518.0,16.9600,17.9400,16.9100,17.7300,177655.0,16.8100,17.1000,16.5700,16.9300
3,AAP,13.9000,14.4667,13.3833,13.4000,371100.0,14.2834,14.2834,13.9000,13.9000,...,1464300.0,17.3033,17.3033,16.7000,16.7000,0.0,17.1830,17.5500,17.0000,17.4300
4,AAT,21.2500,22.0000,21.1800,21.5300,15536889.0,21.3100,21.4500,21.1600,21.1600,...,55327.0,20.4300,20.8000,20.0900,20.8000,142910.0,20.6900,20.7100,20.2000,20.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,ZUMZ,12.4350,12.5750,10.2500,10.2750,6793424.0,12.2550,12.8450,11.7000,12.5500,...,372914.0,34.8200,35.8900,34.5500,35.4600,138372.0,NaN,NaN,NaN,NaN
3758,ZUO,20.0000,21.8500,19.1200,20.0000,13102419.0,20.6000,20.8200,19.6000,19.8000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3759,ZX,5.8500,6.2000,5.6000,6.0000,4987284.0,6.2500,6.4000,5.9900,6.0000,...,55560.0,3.4700,3.5700,3.4300,3.5200,3849.0,3.4600,3.5500,3.3600,3.5300
3760,ZYME,13.0000,14.2500,13.0000,13.5000,953887.0,13.0700,13.2000,13.0000,13.2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_main = df.drop(df.columns[close_day_0:volume_day_261], axis=1, inplace=False)
df_main

,Symbol,DaysBetterThanSP,daysProfit,daysProfitGrouped,exactDiffernce,Year,Month,Day,dayOfWeek,volumeDay261,...,usableCEOAge,usableCEOGender,usablePresidentAge,usablePresidentGender,FoundingDateGrouped,yearDifferenceGrouped,Profitable,Safe,HomeRunDay,HomeRun
0,A,122,249,200+,NaN,1999,11,18,3,8351788.0,...,Blank,Blank,Blank,Blank,1951 - 2000,Unknown,1,0,77.0,1
1,AAC,131,262,200+,232.0,2014,10,2,3,295123.0,...,40 - 49,male,Blank,Blank,2011 - present,Unknown,1,1,NaN,0
2,AAOI,125,262,200+,6054.0,2013,9,26,3,230281.0,...,Blank,Blank,Blank,Blank,1951 - 2000,Unknown,1,0,NaN,0
3,AAP,128,261,200+,NaN,2001,11,29,3,553200.0,...,Blank,Blank,Blank,Blank,1901 - 1950,Unknown,1,0,NaN,0
4,AAT,123,127,100 - 149,181.0,2011,1,13,3,87872.0,...,Blank,Blank,Blank,Blank,2001 - 2010,Unknown,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,ZUMZ,139,261,200+,NaN,2005,5,6,4,NaN,...,Blank,Blank,Blank,Blank,1951 - 2000,Unknown,1,1,234.0,1
3758,ZUO,5,7,0 - 49,NaN,2018,4,12,3,NaN,...,NaN,mostly_male,Unknown,Unknown,2001 - 2010,Unknown,0,0,NaN,0
3759,ZX,102,25,0 - 49,1035.0,2011,5,12,3,22850.0,...,Unknown,Unknown,Unknown,Unknown,2001 - 2010,Unknown,0,0,NaN,0
3760,ZYME,115,19,0 - 49,NaN,2017,4,28,4,NaN,...,Unknown,Unknown,Unknown,Unknown,2001 - 2010,Unknown,0,0,NaN,0


In [51]:
print(df_prices.shape)
print(df_main.shape)

(3762, 1310)
(3762, 355)
